## Set Global Seed 

In [1]:
import os
import random
import numpy as np
import torch
import transformers

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 260615
set_all_seeds(seed)

print("The global seed " + str(seed))

The global seed 260615


## Hyperparameters

In [2]:
# LANGUAGE

_LANGUAGE_         = 'es'
_DATASET_          = '2017'

In [3]:
# MODEL CLASSIFICATION

_PRETRAINED_LM_    = 'vicgalle/xlm-roberta-large-xnli-anli'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.PrefixTuningConfig()
_MAX_SEQ_LEN_      = 150

In [4]:
# TRAIN

_OUTPUT_DIR_       = 'checkPointsPrefixTuningNLI'
_LOGGING_STEPS_    = 50
_NUM_AUTHORS_      = 512
_K_FOLD_CV_        = 5
_NO_GPUS_          = 1
_BATCH_SIZE_       = int(8 / _NO_GPUS_)
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-5

# PREDICTIONS

_PRED_DIR_         = 'PrefixTuningNLI'

## Other parameters

In [5]:
# LABEL DICTONARIES -----------------------------------------------------------------------

# 2015

age_dict  = {'18-24': 0, '25-34': 1, '35-49': 2, '50-XX': 3}
ageEN_hyp = {0: '18-24', 1: '25-34', 2: '35-49', 3: '50-XX'}
ageES_hyp = {0: 'La edad de esta persona es entre 18 y 24 años', 
             1: 'La edad de esta persona es entre 25 y 34 años', 
             2: 'La edad de esta persona es entre 35 y 49 años', 
             3: 'La edad de esta persona es más de 50 años'}

# 2017

gender_dict    = {'female': 0, 'male':   1}
varietyEN_dict = {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}
varietyES_dict = {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}  

genderEN_hyp  = {0: 'I’m a female', 1: 'I’m a male'}
genderES_hyp  = {0: 'Mi nombre es María', 1: 'Mi nombre es José'}

# 2019

bots_dict  = {'human': 0, 'bot': 1}
botsEN_hyp = {0: 'This is a text from a person', 1: 'This is a text from a machine'}
botsES_hyp = {0: 'Humano', 1: 'Bot'}

# 2020 

fakeNews_dict  = {'0': 0, '1': 1}
fakeNewsEN_hyp = {0: 'This author is a normal user', 1: 'This author spreads fake news'}
fakeNewsES_hyp = {0: 'Este autor es un usuario normal', 1: 'Este autor publica noticias falsas'}

# 2021

hateSpeech_dict  = {'0': 0, '1': 1}
hateSpeechEN_hyp = {0: 'This text does not contain hate speech', 1: 'This text expresses prejudice and hate speech'}
hateSpeechES_hyp = {0: 'Este texto es moderado, respetuoso, cortés y civilizado', 1: 'Este texto expresa odio o prejuicios'}

In [6]:
# SET LANGUAGE DICTIONARIES --------------------------------------------------

if _LANGUAGE_ == 'en':
    age_hyp        = ageEN_hyp
    gender_hyp     = genderEN_hyp
    variety_dict   = varietyEN_dict
    fakeNews_hyp   = fakeNewsEN_hyp
    hateSpeech_hyp = hateSpeechEN_hyp
    bots_hyp       = botsEN_hyp 

elif _LANGUAGE_ == 'es':
    age_hyp        = ageES_hyp
    gender_hyp     = genderES_hyp
    variety_dict   = varietyES_dict
    fakeNews_hyp   = fakeNewsES_hyp
    hateSpeech_hyp = hateSpeechES_hyp
    bots_hyp       = botsES_hyp
    
    
# SET LANGUAGE AND DATASET PARAMETERS ----------------------------------------
    
if   _DATASET_ == '2015':
    label_idx  = 2
    class_dict = age_dict
    label_name = 'age'
    label_hyp  = age_hyp
    
elif _DATASET_ == '2017':
    label_idx  = 1
    class_dict = gender_dict
    label_name = 'gender'
    label_hyp  = gender_hyp
    
elif _DATASET_ == '2019':
    label_idx  = 1
    class_dict = bots_dict
    label_name = 'bots'
    label_hyp  = bots_hyp
    
elif _DATASET_ == '2020':
    label_idx  = 1
    class_dict = fakeNews_dict
    label_name = 'fakeNews'
    label_hyp  = fakeNews_hyp
    
elif _DATASET_ == '2021':
    label_idx  = 1
    class_dict = hateSpeech_dict
    label_name = 'hateSpeech'
    label_hyp  = hateSpeech_hyp
    

In [7]:
from transformers import AutoTokenizer, PretrainedConfig

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

config             = PretrainedConfig.from_pretrained(_PRETRAINED_LM_)
nli_label2id       = config.label2id
is_encoder_decoder = config.is_encoder_decoder

You are using a model of type xlm-roberta to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


## Datasets

In [8]:
# GET AUTHORS AND LABELS -----------------------------------------------------

from tools.DataLoaders import BasePAN

baseTrain  = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'train',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)

baseTest   = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'test',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)

In [9]:
# GET K-FOLD SPLITS -----------------------------------------------------

crossVal_splits = baseTrain.cross_val(_K_FOLD_CV_, _NUM_AUTHORS_)

crossVal_splits[0][0]

['28eec74ab394581345f4750efea950cd',
 'c1e059d44ce910d22592011319044e38',
 'd944d9010adae75a63dbf3f951e4a1c6',
 'a2f9698213251f7f32f210369598ba75',
 'dbbcafcfc01c78ef38b593086b01d5ee',
 '20ba60e76325617a7f986688b8d27b0e',
 'dd8fc5e60bdedd21f104594171c0461e',
 'a55f7c8283b14be7d8aeb5e8dac42892',
 'c30a02e9bb8e73ba1fbdd20b7f8745d7',
 '6e035be020c826293c084d0562c15a3a',
 '7a304a60e096b953513ed6ec3fc61d67',
 '6ecd0dad2670d92e419c32189fcc4c0',
 'a3fb3063a98754251bd1c21af3a36378',
 '9321fa54f5fb4acea06da6f3d5936886',
 '5341c6e38c6ff6d6a3dc11eac05a08de',
 '763e38e7eceb818583dec8ee3d2db100',
 '1ae8c47048b807de924559075a372a4c',
 '1cca51f7f4fd8d35cdcdc5f5e563dbda',
 'c45d9059f60848372feb21a0eb1ab04d',
 '462447b733ea398373e911624c65c467',
 '77e3326399b5fae3573f6a0a7442cf22',
 '1ed4440b5bc2800b86533e09fec03c9f',
 'c6f2552b09922bc2cacb9738e80200a3',
 '50669b81a0db1da851dd51687e304582',
 '1c02e48303eadb88aa8185a8ea158171',
 'd9028ea9164a22bde5ec3dec8665adae',
 'da8b39d656aeeff063373f45a777394f',
 '

In [10]:
# GET TWEETS -----------------------------------------------------

baseTrain.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 168000



## Training and Testing

In [11]:
from tools.DataLoaders import DatasetPANnli

baseTest.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)

Test = DatasetPANnli(baseTest)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 112000



In [12]:
from transformers import TrainingArguments

samples = 4 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 5,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [ ]:
from transformers import AutoAdapterModel, AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossValnli
from transformers import AdapterTrainer
from tools.Testing import compute_author_predictions_nli, compute_author_predictions_nli_LR
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

f1s_soft = []
f1s_hard = []

f1s_soft_LR = []
f1s_hard_LR = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossValnli(baseTrain, authors_train)
    Val   = DatasetCrossValnli(baseTrain, authors_val)
    
    
    # initialize model ---------------------------------------------------
    

    static_head_model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_)
    static_head_model.add_adapter(task, config = _ADAPTER_CONFIG_) 
    static_head_model.save_adapter('aux_adapter_'+_PRED_DIR_, task)

    model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)
    model.load_adapter('aux_adapter_'+_PRED_DIR_, load_as=task)
    #model.load_adapter('pretrained_adapter_NLI', load_as=task)

    model.set_active_adapters(task)
    model.train_adapter(task)


    # create trainer and train -------------------------------------------

    trainer = AdapterTrainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    
    ignore_keys = None
    if is_encoder_decoder:
        ignore_keys = ['encoder_last_hidden_state']

    results            = trainer.predict(Test , ignore_keys = ignore_keys)
    author_predictions = compute_author_predictions_nli(baseTest, results.predictions, task, len(class_dict), nli_label2id)

    # report metrics 

    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )
    
    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])


    # get predictions with Logistic Regression----------------------------

    resultsTrain = trainer.predict(Train, ignore_keys = ignore_keys)
    author_predictions_LR = compute_author_predictions_nli_LR(Train, baseTest, resultsTrain.predictions, results.predictions, task, len(class_dict))
    
    f1s_soft_LR.append( f1_score(author_predictions_LR['true'], author_predictions_LR['pred_soft'], average = 'macro') )
    f1s_hard_LR.append( f1_score(author_predictions_LR['true'], author_predictions_LR['pred_hard'], average = 'macro') )
    
    # report metrics 

    report_LR = {'soft': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_hard'], digits=4)}

    print("Results with split " + str(split + 1) + " using LOGISTIC REGRESSION:\n")
    print("soft voting:\n", report_LR['soft'], '\n')
    print("hard voting:\n", report_LR['hard'])

     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/test_split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)
    
    with open(DIR + 'predictions_LR.pickle', 'wb') as f:
        pickle.dump(author_predictions_LR, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
        
    with open(DIR + 'report_LR.txt', 'w') as f:
        f.write("soft voting:\n" + report_LR['soft'] + '\n\n')
        f.write("hard voting:\n" + report_LR['hard'])
    


Some weights of the model checkpoint at vicgalle/xlm-roberta-large-xnli-anli were not used when initializing XLMRobertaAdapterModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
- This IS expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at vicgalle/xlm-roberta-large-xnli-anli and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to b

Step,Training Loss
5120,0.662100
10240,0.609200
15360,0.593700
20480,0.580800
25600,0.571300
30720,0.561500
35840,0.555400
40960,0.548300


Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-208000] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-1000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-1000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-1000/gender/pytorch_adapter.bin
Configuration sav

Module weights saved in checkPointsPrefixTuningNLI/checkpoint-6000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-6000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-6000/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-6000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-6000/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-3500] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-6500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-6500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-6500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-6500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-6500/gender/pytorch_mod

Module weights saved in checkPointsPrefixTuningNLI/checkpoint-11500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-11500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-11500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-9000] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-12000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-12000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-12000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-12000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-12000/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-12000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-12000/gende

Configuration saved in checkPointsPrefixTuningNLI/checkpoint-17000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-17000/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-14500] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-17500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-17500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-15000] 

Module weights saved in checkPointsPrefixTuningNLI/checkpoint-22500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-20000] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-23000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-23000/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-20500] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoi

Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-25500] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-28500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-28500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-26000] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-29000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-29000/gender/adapter_config.

Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-34000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-34000/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-31500] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-34500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-34500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-34500/gender/pytorch_adapter.bin
C

Configuration saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-39500/gender/pytorch_model_head.bin
Deleting older checkpoint [checkPointsPrefixTuningNLI/checkpoint-37000] due to args.save_total_limit
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-40000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-40000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-40000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-40000/gende

In [16]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])

Soft results:  [0.79707287 0.80908847 0.79379599 0.79280598 0.80738972]

Hard results:  [0.77554967 0.79675748 0.77909846 0.78641539 0.79281898]


Soft statistics: 
	[avg, std]: [0.8000306038782099, 0.0068704493968496485]

Hard statistics: 
	[avg, std]: [0.7861279969050816, 0.007989408678768545]


In [17]:
# report statistics

print('Soft results: ', f1s_soft_LR)
print('\nHard results: ', f1s_hard_LR)

f1s_soft_LR = np.array(f1s_soft_LR)
f1s_hard_LR = np.array(f1s_hard_LR)

FewShot_Results_LR = {'soft': [f1s_soft_LR.mean(), f1s_soft_LR.std()], 'hard': [f1s_hard_LR.mean(), f1s_hard_LR.std()]}

print('\n\nSoft statistics with LOGISTIC REGRESSION: ')
print('\t[avg, std]:', FewShot_Results_LR['soft'])

print('\nHard statistics with LOGISTIC REGRESSION: ')
print('\t[avg, std]:', FewShot_Results_LR['hard'])

Soft results:  [0.80320703 0.8149815  0.79856886 0.79571053 0.81170768]

Hard results:  [0.77962176 0.79994121 0.78235206 0.78490771 0.79312243]


Soft statistics with LOGISTIC REGRESSION: 
	[avg, std]: [0.804835121168187, 0.007420995794740641]

Hard statistics with LOGISTIC REGRESSION: 
	[avg, std]: [0.7879890331064306, 0.007489826995218846]


## Training with all

In [11]:
from tools.DataLoaders import DatasetPANnli

baseTest.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_, NLI=True, label_hyp=label_hyp, nli_label2id=nli_label2id)

Test  = DatasetPANnli(baseTest)
Train = DatasetPANnli(baseTrain)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 112000



In [12]:
from transformers import TrainingArguments

samples = 4 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 5,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [ ]:
from transformers import AutoAdapterModel, AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossValnli
from transformers import AdapterTrainer
from tools.Testing import compute_author_predictions_nli, compute_author_predictions_nli_LR
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

# initialize model ---------------------------------------------------

static_head_model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_)
static_head_model.add_adapter(task, config = _ADAPTER_CONFIG_) 
static_head_model.save_adapter('aux_adapter_'+_PRED_DIR_, task)

model = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)
model.load_adapter('aux_adapter_'+_PRED_DIR_, load_as=task)
#model.load_adapter('pretrained_adapter_NLI', load_as=task)

model.set_active_adapters(task)
model.train_adapter(task)


# create trainer and train -------------------------------------------

trainer = AdapterTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = Train,
)
trainer.args._n_gpu = _NO_GPUS_

trainer.train()


# get predictions ----------------------------------------------------


ignore_keys = None
if is_encoder_decoder:
    ignore_keys = ['encoder_last_hidden_state']

results            = trainer.predict(Test , ignore_keys = ignore_keys)
author_predictions = compute_author_predictions_nli(baseTest, results.predictions, task, len(class_dict), nli_label2id)

# report metrics 

report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

print("Results:\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])


# get predictions with Logistic Regression----------------------------


resultsTrain = trainer.predict(Train, ignore_keys = ignore_keys)
author_predictions_LR = compute_author_predictions_nli_LR(baseTrain, baseTest, resultsTrain.predictions, results.predictions, task, len(class_dict))

# report metrics 

report_LR = {'soft': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_soft'], digits=4), 
           'hard': classification_report(author_predictions_LR['true'], author_predictions_LR['pred_hard'], digits=4)}

print("Results using LOGISTIC REGRESSION:\n")
print("soft voting:\n", report_LR['soft'], '\n')
print("hard voting:\n", report_LR['hard'])



Some weights of the model checkpoint at vicgalle/xlm-roberta-large-xnli-anli were not used when initializing XLMRobertaAdapterModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at vicgalle/xlm-roberta-large-xnli-anli and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to b

Step,Training Loss
5120,0.654000


Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-500
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/head_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-500/gender/pytorch_model_head.bin
Saving model checkpoint to checkPointsPrefixTuningNLI/checkpoint-1000
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-1000/gender/adapter_config.json
Module weights saved in checkPointsPrefixTuningNLI/checkpoint-1000/gender/pytorch_adapter.bin
Configuration saved in checkPointsPrefixTuningNLI/checkpoint-1000/gender/head_config.json
Module weights saved in check

In [17]:

print("Results:\n")
print("soft voting:\n", report['soft'], '\n')
print("hard voting:\n", report['hard'])

Results:

soft voting:
               precision    recall  f1-score   support

           0     0.8180    0.8636    0.8402      1400
           1     0.8555    0.8079    0.8310      1400

    accuracy                         0.8357      2800
   macro avg     0.8368    0.8357    0.8356      2800
weighted avg     0.8368    0.8357    0.8356      2800
 

hard voting:
               precision    recall  f1-score   support

           0     0.7856    0.8714    0.8263      1400
           1     0.8557    0.7621    0.8062      1400

    accuracy                         0.8168      2800
   macro avg     0.8206    0.8168    0.8162      2800
weighted avg     0.8206    0.8168    0.8162      2800



In [18]:

print("Results using LOGISTIC REGRESSION:\n")
print("soft voting:\n", report_LR['soft'], '\n')
print("hard voting:\n", report_LR['hard'])


Results using LOGISTIC REGRESSION:

soft voting:
               precision    recall  f1-score   support

           0     0.8439    0.8264    0.8351      1400
           1     0.8300    0.8471    0.8385      1400

    accuracy                         0.8368      2800
   macro avg     0.8369    0.8368    0.8368      2800
weighted avg     0.8369    0.8368    0.8368      2800
 

hard voting:
               precision    recall  f1-score   support

           0     0.8124    0.8350    0.8235      1400
           1     0.8303    0.8071    0.8185      1400

    accuracy                         0.8211      2800
   macro avg     0.8213    0.8211    0.8210      2800
weighted avg     0.8213    0.8211    0.8210      2800



In [ ]:
model.save_adapter('Pretrained_Adapters_NLI/'+_DATASET_+_LANGUAGE_+'_'+_PRED_DIR_, task)